# Dots and Boxes in Python
Dots and Boxes is a simple Strategy game developed in the 1880s in France.
This program will attempt to recreate it in Python and apply the Decision Tree algorithm to the results, with a limited ply-depth to ensure that the program does not blow up Google Colab.

## Dots and Boxes logic
The game dots and boxes is a game that involves conquering territory from the opposing player. When all four edges are placed around a box, the box is considered "Won" on behalf of the entity that took the box.

A box is only considered "Won" when the final edge is filled in. The entity that filled in the final edge wins the box in question.

There is a line of logic that would allow for a minimum of a draw every time, but looks too hard to implement. This was decided against. Instead, the computer can choose between moves that assess the box, and neighbouring boxes directly. If a box has one available placement, it takes it, and tries not to place a line if the box has two sides filled, as this would make it too easy for the opposing player.

In [15]:
class SearchTreeNode:
  def __init__(self, game_board, current_player, ply=0, max_depth=3): # Added a maximum depth of 5 to increase performance
    self.children = []
    self.value_is_assigned = False
    self.ply_depth = ply
    self.current_board = game_board
    self.move_for = current_player

    if self.ply_depth < max_depth and self.current_board.state_of_board() == "U": # Still using Won or Not won to decide state
      self.generate_children(max_depth)
    else:
      player_score, computer_score = self.current_board.count_score()
      self.value = computer_score - player_score
      self.value_is_assigned = True

  def min_max_value(self, max_depth):
    if self.value_is_assigned:
      return self.value
    for child in self.children:
      child.min_max_value(max_depth)
    if self.move_for == 2: # Computer Move
      if not self.children:
        self.value = -999
      else:
        self.value = max(child.value for child in self.children)
    else:
      if not self.children:
        self.value = 999
      else:
        self.value = min(child.value for child in self.children)
    self.value_is_assigned = True
    return self.value

  def generate_children(self, max_depth):
    for next_board in self.current_board.all_possible_moves():
      self.children.append(SearchTreeNode(next_board, 3 - self.move_for, ply = self.ply_depth +1, max_depth = max_depth))

In [16]:
'''Map of the Game board for reference. It's long for easier deciphering of indices.'''
#      [0,1]     [0,3]     [0,5]
# [1,0]     [1,2]     [1,4]     [1,6]
#      [2,1]     [2,3]     [2,5]
# [3,0]     [3,2]     [3,4]     [3,6]
#      [4,1]     [4,3]     [4,5]
class DotsAndBoxes:
  def __init__(self, newBoard = None, current_player = 1, player_score = 0, computer_score = 0): # Implement Game State for algorithm.
    self.current_player = current_player # For copying the board, this needs to be reinitialized.
    self.player_score = player_score
    self.computer_score = computer_score
    if newBoard is None:
      self.game_board = [
        ["*", " ", "*", " ", "*", " ", "*"],
        [" ", "0", " ", "0", " ", "0", " "],
        ["*", " ", "*", " ", "*", " ", "*"],
        [" ", "0", " ", "0", " ", "0", " "],
        ["*", " ", "*", " ", "*", " ", "*"]
      ]
    else: # Otherwise initialize this game board instance.
      self.game_board = [row[:] for row in newBoard] # copy
    self.state = self.state_of_board() # No longer causing a crash

  def display_board(self):
    for row in self.game_board:
      print(''.join(row))

  def state_of_board(self): # All that needed to be assessed here is whether the game is won or not.
    if self.return_available_moves():
      return "U" # Unfinished
    else:
      return "D"

  def return_available_moves(self):
    remaining_moves = []
    for row_index in range(len(self.game_board)):
      for col_index in range(len(self.game_board[row_index])):
        if self.game_board[row_index][col_index] == " ":
          remaining_moves.append((row_index, col_index))
    return remaining_moves

  def all_possible_moves(self): # Updates the board, return available moves.
    remaining_moves = self.return_available_moves()
    all_moves = []
    for move in remaining_moves:
      new_board = DotsAndBoxes( # Create board with move
          [row[:] for row in self.game_board],
          self.current_player,
          self.player_score,
          self.computer_score
      )
      new_board.add_line(move[0], move[1])
      boxes_completed = new_board.check_if_box_completed(move[0], move[1])
      if boxes_completed == 0: # Logic to change player if box is complete
        new_board.change_player()
      all_moves.append(new_board)
    return all_moves

  def check_if_box_completed(self, row, col): # Checks if placing a line completes more than one box. Checks run after every placed line.
    boxes_completed = 0
    if row % 2 == 0:# Checks horizontal line. Block handles box above.
      if row > 0:
        r, c = row - 1, col
        if self.game_board[r][c] == "0" and self.count_lines_around_box(r, c) == 4:
          self.game_board[r][c] = str(self.current_player)
          if self.current_player == 1:
            self.player_score += 1
          else:
            self.computer_score += 1
          boxes_completed += 1
      if row < len(self.game_board) - 1: # Block handles box below.
        r, c = row + 1, col
        if self.game_board[r][c] == "0" and self.count_lines_around_box(r, c) == 4:
          self.game_board[r][c] = str(self.current_player)
          if self.current_player == 1:
            self.player_score += 1
          else:
            self.computer_score += 1
          boxes_completed += 1

    else: # The following block handles Vertical box checks
      if col > 0: # Check left box
        r, c = row, col - 1
        if self.game_board[r][c] == "0" and self.count_lines_around_box(r, c) == 4:
          self.game_board[r][c] = str(self.current_player)
          if self.current_player == 1:
            self.player_score += 1
          else:
            self.computer_score += 1
          boxes_completed += 1
      if col < len(self.game_board[row]) - 1: # Check right box
        r, c = row, col + 1
        if self.game_board[r][c] == "0" and self.count_lines_around_box(r, c) == 4:
          self.game_board[r][c] = str(self.current_player)
          if self.current_player == 1:
            self.player_score += 1
          else:
            self.computer_score += 1
          boxes_completed += 1
    return boxes_completed

  def count_lines_around_box(self, row, col): # Refactored to return numbers
    if self.game_board[row][col] not in ['0', '1', '2']:
      return 0
    count = 0
    if row > 0 and self.game_board[row - 1][col] == '-': # Top
      count += 1
    if row < len(self.game_board) - 1 and self.game_board[row + 1][col] == '-':
      count += 1
    if row > 0 and self.game_board[row - 1][col] == '-': # Check Left
      count += 1
    if col < len(self.game_board[row]) - 1 and self.game_board[row][col + 1] == "-": # Check right
      count += 1
    return count

  def count_score(self):
    self.player_score = sum(row.count("1") for row in self.game_board)
    self.computer_score = sum(row.count("2") for row in self.game_board)
    return self.player_score, self.computer_score

  def change_player(self):
    self.current_player = 3 - self.current_player

  def player_move(self):
    remaining_moves = self.return_available_moves()
    player_score, computer_score = self.count_score()
    print(f"Player Score: {player_score}")
    print(f"Computer Score: {computer_score}")
    print(f"Remaining Moves: {remaining_moves}")
    row = int(input("Please enter a row for the move (0-4): "))
    col = int(input("Please enter a column for the move (0-4): "))
    if (row, col) not in remaining_moves:
      print("Invalid move. Please try again.")
      return False
    self.add_line(row, col)
    boxes_completed = self.check_if_box_completed(row, col)
    if boxes_completed == 0:
      self.change_player()
    else:
      print(f"Box complete! Take another turn.{boxes_completed}")
    return True

  def find_completable_box(self):
    for row in range(len(self.game_board)):
      for col in range(len(self.game_board[row])):
        if self.game_board[row][col] == "0":
          if self.count_lines_around_box(row, col) == 3:
            if row > 0 and self.game_board[row - 1][col] == " ":
              return row - 1, col
            if row < len(self.game_board) - 1 and self.game_board[row + 1][col] == " ":
              return row + 1, col
            if col > 0 and self.game_board[row][col - 1] == " ":
              return row, col - 1
            if col < len(self.game_board[row]) - 1 and self.game_board[row][col + 1] == " ":
              return row, col + 1
    return None

  def add_line(self, row, col):
    self.game_board[row][col] = "-"

  def computer_move(self): # It's this. Definitely this.
    move = self.find_completable_box() # First Priority.
    if not move:
      move = self.find_safe_move() # Second Priority.

    if not move:
      search_tree = SearchTreeNode(self, self.current_player, max_depth = 2)
      search_tree.min_max_value(2)
      best_value = float("-inf")
      best_board = None
      for child in search_tree.children:
        if child.value > best_value:
          best_value = child.value
          best_board = child.current_board
      if best_board:
        for row in range(len(self.game_board)):
          for col in range(len(self.game_board[row])):
            if(self.game_board[row][col] == " " and
               best_board.game_board[row][col] == "-"):
              move = (row, col)
      if not move:
        moves = self.return_available_moves()
        if moves:
          move = moves[0]
        else:
          return
    print(f"Computer made a move at: {move}")
    self.add_line(move[0], move[1])
    boxes_completed = self.check_if_box_completed(move[0], move[1])
    if boxes_completed > 0:
      self.computer_move()
    else:
      self.change_player()

  def find_safe_move(self): # Do not make a move that puts three sides on a box. It gives away the box to the next player. That's all this does.
    safe_moves = []
    for move in self.return_available_moves():
      gives_away_box = False
      test_board = [row[:] for row in self.game_board]
      test_board[move[0]][move[1]] = "-"
      box_positions = []
      if move[0] % 2 == 0:
        if move[0] > 0:
          box_positions.append((move[0] - 1, move[1]))
        if move[0] < len(test_board) - 1:
          box_positions.append((move[0] + 1, move[1]))
      else:
        if move[1] > 0:
          box_positions.append((move[0], move[1] - 1))
        if move[1] < len(test_board[move[0]]) - 1:
          box_positions.append((move[0], move[1] + 1))
      for box_row, box_col in box_positions:
        if (0 <= box_row < len(test_board) and
            0 < box_col < len(test_board[box_row]) and
            test_board[box_row][box_col] == 0):
          line_count = 0
          if box_row > 0 and (test_board[box_row-1][box_col] == "-"):
            line_count += 1
          if box_row < len(test_board)-1 and (test_board[box_row+1][box_col] == "-"):
            line_count += 1
          if box_col > 0 and (test_board[box_row][box_col-1] == "-"):
            line_count += 1
          if box_col < len(test_board[box_row])-1 and (test_board[box_row][box_col+1] == "-"):
            line_count += 1

          if line_count == 3:
            gives_away_box = True
      if not gives_away_box:
        safe_moves.append(move)
    if safe_moves:
      return safe_moves[0] # Select the first available move from moves.
    else:
      return None

  def play_game(self): # Not where the error is happening.
    self.display_board()
    while self.state_of_board() == "U":
      if self.current_player == 1:
        valid_move = self.player_move()
        while not valid_move:
          valid_move = self.player_move()
      else:
        print("Computer made a move.")
        self.computer_move()
      self.display_board()
      player_score, computer_score = self.count_score()
      print(f"Current Score - Player:{player_score}, Computer: {computer_score}")
    print(f"Final Score - Player:{player_score}, Computer: {computer_score}")
    if player_score > computer_score:
      print("Congratulations! You won!")
    elif player_score < computer_score:
      print("Computer wins!")
    else:
      print("It's a draw!")

In [17]:
cb = DotsAndBoxes()

In [18]:
game = DotsAndBoxes()
for g in game.all_possible_moves():
  g.display_board()

*-* * *
 0 0 0 
* * * *
 0 0 0 
* * * *
* *-* *
 0 0 0 
* * * *
 0 0 0 
* * * *
* * *-*
 0 0 0 
* * * *
 0 0 0 
* * * *
* * * *
-0 0 0 
* * * *
 0 0 0 
* * * *
* * * *
 0-0 0 
* * * *
 0 0 0 
* * * *
* * * *
 0 0-0 
* * * *
 0 0 0 
* * * *
* * * *
 0 0 0-
* * * *
 0 0 0 
* * * *
* * * *
 0 0 0 
*-* * *
 0 0 0 
* * * *
* * * *
 0 0 0 
* *-* *
 0 0 0 
* * * *
* * * *
 0 0 0 
* * *-*
 0 0 0 
* * * *
* * * *
 0 0 0 
* * * *
-0 0 0 
* * * *
* * * *
 0 0 0 
* * * *
 0-0 0 
* * * *
* * * *
 0 0 0 
* * * *
 0 0-0 
* * * *
* * * *
 0 0 0 
* * * *
 0 0 0-
* * * *
* * * *
 0 0 0 
* * * *
 0 0 0 
*-* * *
* * * *
 0 0 0 
* * * *
 0 0 0 
* *-* *
* * * *
 0 0 0 
* * * *
 0 0 0 
* * *-*


In [19]:
cb.display_board()

* * * *
 0 0 0 
* * * *
 0 0 0 
* * * *


In [20]:
st = SearchTreeNode(cb,1)

In [21]:
game = DotsAndBoxes()
game.play_game()

* * * *
 0 0 0 
* * * *
 0 0 0 
* * * *
Player Score: 0
Computer Score: 0
Remaining Moves: [(0, 1), (0, 3), (0, 5), (1, 0), (1, 2), (1, 4), (1, 6), (2, 1), (2, 3), (2, 5), (3, 0), (3, 2), (3, 4), (3, 6), (4, 1), (4, 3), (4, 5)]
Please enter a row for the move (0-4): 1
Please enter a column for the move (0-4): 6
* * * *
 0 0 0-
* * * *
 0 0 0 
* * * *
Current Score - Player:0, Computer: 0
Computer made a move.
Computer made a move at: (0, 1)
*-* * *
 0 0 0-
* * * *
 0 0 0 
* * * *
Current Score - Player:0, Computer: 0
Player Score: 0
Computer Score: 0
Remaining Moves: [(0, 3), (0, 5), (1, 0), (1, 2), (1, 4), (2, 1), (2, 3), (2, 5), (3, 0), (3, 2), (3, 4), (3, 6), (4, 1), (4, 3), (4, 5)]
Please enter a row for the move (0-4): 2
Please enter a column for the move (0-4): 3
*-* * *
 0 0 0-
* *-* *
 0 0 0 
* * * *
Current Score - Player:0, Computer: 0
Computer made a move.
Computer made a move at: (0, 3)
*-*-* *
 0 0 0-
* *-* *
 0 0 0 
* * * *
Current Score - Player:0, Computer: 0
Player Sco